In [3]:
using DataFrames
using JuMP, Gurobi

In [4]:
# number of teams
N = 20
# number of weeks
W = N-1
# number of days per weeks
D = 7;

In [5]:
function GetC()
    C = zeros((D*W, D*W))

    for t in 1:(W*D)
        for t_ in 1:(W*D)
            if t < t_
                C[t, t_] = (t_ - t)^(-1)
            end
        end
    end

    return C
end

GetC (generic function with 1 method)

In [27]:
function Optimize4WeeksAndFreeze(X_old, Y_old)

    # create model
    model = Model(Gurobi.Optimizer)

    # set time limit
    set_optimizer_attribute(model, "TimeLimit", 300)

    # PARAMETERS
    C = GetC()
    N = 20 # teams
    W = 4 # weeks
    D = 7 # days

    println(1)

    # VARIABLES
    @variable(model, x[i = 1:N, j = 1:N, w = 1:W, d = 1:D], Bin) # 1 if team i plays team j on day d of week w, 0 otherwise
    @variable(model, a[i = 1:N, j = 1:N, k = 1:N, w = 1:W, d = 1:D, d_ = 1:D] >= 0) # will be pushed to be binary
    @variable(model, y[i = 1:N, j = 1:N], Bin) # 1 if team i played team j on w, 0 otherwise

    println(2)

    # OBJECTIVE FUNCTION
    @objective(model, Min, sum(C[7 * w + d, 7 * (w+1) + d_] * a[i, j, k, w, d, d_] for i in 1:N, 
        j in 1:N, k in 1:N, w in 1:W-1, d in 1:D, d_ in 1:D))

    println(3)

    # CONSTRAINTS
    # linearize objective function
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = 1:W-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] <= x[i, j, w, d])
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = 1:W-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] <= x[i, k, w+1, d])
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = 1:W-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] >= x[i, j, w, d] + x[i, k, w+1, d] - 1)

    println(4)

    # teams can never play against themselves
    @constraint(model, [i = 1:N, w = 1:W, d = 1:D], x[i, i, w, d] == 0)

    println(5)

    # if A plays against B, then B plays against A
    @constraint(model, [i = 1:N, j = 1:N, w = 1:W, d = 1:D], x[i, j, w, d] == x[j, i, w, d])

    println(6)

    # each team can only play at most one game per week
    @constraint(model, [i = 1:N, w = 1:W], sum(x[i, j, w, d] for j in 1:N, d in 1:D) <= 1)

    println(7)

    # each team plays Wend - Win + 1 games in total
    @constraint(model, [i = 1:N], sum(x[i, j, w, d] for j in 1:N, w in 1:W, d in 1:D) == W)

    println(8)

    # each game is against a different team, team play against each other team at most once
    @constraint(model, [i = 1:N, j = 1:N], sum(x[i, j, w, d] for w in 1:W, d in 1:D) <= 1)

    println(9)

    # if team i plays team j, update Y
    @constraint(model, [i = 1:N, j = 1:N, w = 1:W], y[i, j] >= sum(x[i, j, w, d] for d in 1:D, w = 1:W))

    println(10)

    print(Y_old)

    # if team i played team j in the past, then they can't play again now
    @constraint(model, [i = 1:N, j = 1:N], sum(x[i,j,w,d] for w in 1:W, d in 1:D) <= 1 - Y_old[i, j])

    println(11)

    # cannot plays on days 1,2,3
    @constraint(model, [i = 1:N, j = 1:N, w = 1:W, d = 1:3], x[i, j, w, d] == 0)

    println(12)

    # OPTIMIZE
    optimize!(model)

    println(13)

    # update old X, Y before returning them
    for i in 1:N
        for j in 1:N
            for w in 1:W
                for d in 1:D
                    X_old[i, j, w, d] = value(x[i, j, w, d])
                    Y_old[i,j] = value(y[i,j])
                end
            end
        end
    end

    println(14)

    return X_old, Y_old


end


Optimize4WeeksAndFreeze (generic function with 1 method)

In [26]:
# first time, X_old and Y_old initialized to empty
X_old = zeros(20, 20, 4, 7) # teams are never playing against each other
Y_old = zeros(20, 20)   # teams never played against each other before
;

In [28]:
# optimize for 4 weeks
X, y = Optimize4WeeksAndFreeze(X_old, Y_old)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 300
1


2


3


4
5
6
7
8
9
10
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 


14

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;;; 0.0 0.0 … 0

In [29]:
for i = 1:20
    for j = 1:20
        for w = 1:4
            for d = 1:7
                if X_[i, j, w, d] == 1.0
                    println("Team ", i, " plays team ", j, " on day ", d, " of week ", w)
                end
            end
        end
    end
end

UndefVarError: UndefVarError: X not defined